# CatBoost

In [161]:
from catboost import CatBoostRegressor
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import math
import sklearn

In [162]:
# Inserting features from TRAIN into one dataset

folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)

       average_income_per_grunnkrets  numberOfBusstop categorical_chain_name  \
0                           561700.0             10.0              MCDONALDS   
1                           555720.0             30.0              MCDONALDS   
2                           495900.0             14.0            BURGER KING   
3                           521240.0             11.0            BURGER KING   
4                           585360.0             29.0            BURGER KING   
...                              ...              ...                    ...   
12854                       505640.0             27.0                    NaN   
12855                       396640.0             12.0       GULATING GRUPPEN   
12856                       549120.0             13.0       GULATING GRUPPEN   
12857                       498660.0             35.0                    NaN   
12858                       579780.0             10.0                    NaN   

      categorical_mall_name categorical

In [163]:
# Inserting features from TEST into one dataset

folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)

      average_income_per_grunnkrets  numberOfBusstop categorical_chain_name  \
0                          551600.0             21.0                    NaN   
1                          531240.0             11.0            BURGER KING   
2                          523300.0             11.0            BURGER KING   
3                          591380.0             22.0            BURGER KING   
4                          594020.0             21.0       VULKAN BURGERBAR   
...                             ...              ...                    ...   
8572                       560700.0              9.0       GULATING GRUPPEN   
8573                       593340.0             28.0                    NaN   
8574                       629100.0              6.0       GULATING GRUPPEN   
8575                       590100.0              8.0                    NaN   
8576                       529980.0              4.0                    NaN   

     categorical_mall_name categorical_plaace categ

In [164]:
# Target TRAIN data

target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

target_train = np.log1p(target_train)

In [165]:
catboost_model = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    thread_count=-1,
    max_depth=6,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    bagging_temperature=0.3,
    od_type="Iter"
)


In [166]:
for column in features_train.columns:
    values=features_train[column].values
    for i in range(len(values)):
        if type(values[i]) == float:
            if math.isnan(values[i]):
                values[i]="nan"
                

In [167]:
for column in features_test.columns:
    values=features_test[column].values
    for i in range(len(values)):
        if type(values[i]) == float:
            if math.isnan(values[i]):
                values[i]="nan"

In [168]:
# TRAIN

categorical_features_train = []

for col in features_train.columns:
    if col.startswith("categorical"):
        categorical_features_train.append(col)
        print(col)

Pool_train= Pool(features_train,target_train,cat_features=categorical_features_train)

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


c:\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [169]:
# TEST

categorical_features_test = []

for col in features_test.columns:
    if col.startswith("categorical"):
        categorical_features_test.append(col)

Pool_test= Pool(features_test, cat_features=categorical_features_test)

In [170]:
# Fit and predict

catboost_model.fit(Pool_train)

catboost_predictions = catboost_model.predict(Pool_test)
catboost_predictions = np.expm1(catboost_predictions)




In [171]:
print(catboost_predictions)

[4.92877203 8.30917029 3.7951891  ... 4.90527831 3.3623622  2.54951897]


In [172]:
# Get storeid as first column

stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

# insert predictions as second column

predictions = pd.DataFrame()

predictions['id'] = stores_test_id
predictions['predicted'] = catboost_predictions
predictions.to_csv("../predictions/catboost6.csv", index=False)
print(predictions)

                              id  predicted
0     914206820-914239427-717245   4.928772
1     916789157-916823770-824309   8.309170
2       913341082-977479363-2948   3.795189
3      889682582-889697172-28720   7.534987
4     997991699-998006945-417222  12.620689
...                          ...        ...
8572  917323003-917383529-844309   4.369883
8573  917353379-917411824-845904   3.682447
8574  917072302-917089248-833647   4.905278
8575  916960557-916993161-829908   3.362362
8576   987280891-972040746-45320   2.549519

[8577 rows x 2 columns]


In [173]:
X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

catboost_model2 = CatBoostRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    thread_count=-1,
    max_depth=6,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    bagging_temperature=0.3,
    od_type="Iter"
)

#log transform
y_train = np.log1p(y_train)

# TRAIN

categorical_features_train = []

for col in X_train.columns:
    if col.startswith("categorical"):
        categorical_features_train.append(col)
        print(col)

Pool_train= Pool(X_train,y_train,cat_features=categorical_features_train)

# TEST

categorical_features_test = []

for col in X_test:
    if col.startswith("categorical"):
        categorical_features_test.append(col)

Pool_test= Pool(X_test, cat_features=categorical_features_test)


catboost_model2.fit(Pool_train)
y_pred = catboost_model2.predict(Pool_test)

y_pred = np.expm1(y_pred)

print("Score: ",sklearn.metrics.mean_squared_log_error(y_test, y_pred) ** 0.5)

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


c:\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Score:  0.3176936628936399


In [174]:
0.3187978256079073

0.3187978256079073